<a href="https://colab.research.google.com/github/anitakumar/ml/blob/main/tf_lite_convertor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import numpy as np

model=tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1,input_shape=[1]),
    tf.keras.layers.Dense(units=16,activation=tf.nn.relu),
    tf.keras.layers.Dense(units=1)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(optimizer='sgd',loss='mean_squared_error')

#train the model
model.fit(x=np.array([-1,0,1]), y=np.array([-3,-1,1]),epochs=5)
#save the model
model.save('saved_model_dir.keras')

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - loss: 2.8347
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.7711
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.7080
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 2.6467
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.5884


In [12]:
#covert model to tf-lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)
#

Saved artifact at '/tmp/tmpufz1m0y2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140660262164560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140660262386704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140660127268560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140660126853856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140660127115648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140660299468112: TensorSpec(shape=(), dtype=tf.resource, name=None)


1932

In [16]:
#convert concrete functions
class Squared(tf.Module):
  @tf.function(input_signature=[tf.TensorSpec(shape=[None],dtype=tf.float32)])
  def __call__(self,x):
    return tf.square(x)
model= Squared()
concrete_function=model.__call__.get_concrete_function()
converter=tf.lite.TFLiteConverter.from_concrete_functions([concrete_function])
tflite_model=converter.convert()


